# Assignment Chapter 2 - DEEP LEARNING [Case #4]
Startup Campus, Indonesia - `Artificial Intelligence (AI)` (Batch 7)
* Task: **CLASSIFICATION**
* DL Framework: **PyTorch**
* Dataset: Credit Card Fraud 2023
* Libraries: Pandas/cuDF, Scikit-learn/cuML, Numpy/cuPy
* Objective: Classify credit fraud transactions using Multilayer Perceptron

`PERSYARATAN` Semua modul (termasuk versi yang sesuai) sudah di-install dengan benar.
<br>`CARA PENGERJAAN` Lengkapi baris kode yang ditandai dengan **#TODO**.
<br>`TARGET PORTFOLIO` Peserta mampu mengklasifikasi transaksi fraud menggunakan *Multilayer Perceptron*

### Import Libraries

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 530, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 530 (delta 171), reused 129 (delta 94), pack-reused 269 (from 1)
Receiving objects: 100% (530/530), 170.56 KiB | 933.00 KiB/s, done.
Resolving deltas: 100% (273/273), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.4 MB/s eta 0:00:00
Installing RAPIDS remaining 24.6.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 478.0/478.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import shutil
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

<font color="red">**- - - - MOHON DIPERHATIKAN - - - -**</font>
<br>**Aktifkan GPU sekarang.** Di Google Colab, klik **Runtime > Change Runtime Type**, lalu pilih **T4 GPU**.

### Dataset Loading (CPU vs. GPU)

In [ ]:
from pandas import read_csv as read_by_CPU
from cudf import read_csv as read_by_GPU

In [ ]:
# unzip the file
shutil.unpack_archive('dataset_case_04.zip', '/content/sample_data/', 'zip')

In [ ]:
# TODO: Impor dataset dengan Pandas, gunakan fungsi "read_by_CPU"
%time data_cpu = read_by_CPU('/content/sample_data/creditcard_2023.csv')

CPU times: user 3.75 s, sys: 292 ms, total: 4.04 s
Wall time: 4.06 s


In [ ]:
# Impor dataset dengan cuDF (Pandas di GPU)
%time data_gpu = read_by_GPU('/content/sample_data/creditcard_2023.csv')

CPU times: user 255 ms, sys: 26 ms, total: 281 ms
Wall time: 277 ms


In [ ]:
# TODO: Hilangkan kolom ID
data_gpu = data_gpu.drop(columns=['id'])
data_gpu

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,0.637735,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,0.529808,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,0.690708,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,0.575231,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,0.968046,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568625,-0.833437,0.061886,-0.899794,0.904227,-1.002401,0.481454,-0.370393,0.189694,-0.938153,-1.161847,...,0.167503,0.419731,1.288249,-0.900861,0.560661,-0.006018,3.308968,0.081564,4394.16,1
568626,-0.670459,-0.202896,-0.068129,-0.267328,-0.133660,0.237148,-0.016935,-0.147733,0.483894,-0.210817,...,0.031874,0.388161,-0.154257,-0.846452,-0.153443,1.961398,-1.528642,1.704306,4653.40,1
568627,-0.311997,-0.004095,0.137526,-0.035893,-0.042291,0.121098,-0.070958,-0.019997,-0.122048,-0.144495,...,0.140788,0.536523,-0.211100,-0.448909,0.540073,-0.755836,-0.487540,-0.268741,23572.85,1
568628,0.636871,-0.516970,-0.300889,-0.144480,0.131042,-0.294148,0.580568,-0.207723,0.893527,-0.080078,...,-0.060381,-0.195609,-0.175488,-0.554643,-0.099669,-1.434931,-0.159269,-0.076251,10160.83,1


### Standardization (CPU vs. GPU)

In [ ]:
from sklearn.preprocessing import StandardScaler as StandardScalerCPU
from cuml.preprocessing import StandardScaler as StandardScalerGPU

In [ ]:
ScalerCPU = StandardScalerCPU()
ScalerGPU = StandardScalerGPU()

arbitrary_features = ["V"+str(i+1) for i in range(27)]

In [ ]:
%%time

data_cpu[arbitrary_features] = ScalerCPU.fit_transform(data_cpu[arbitrary_features].values)
data_cpu["Amount"] = ScalerCPU.fit_transform(data_cpu["Amount"].values.reshape(-1, 1)).squeeze()

CPU times: user 168 ms, sys: 753 ms, total: 921 ms
Wall time: 977 ms


In [ ]:
%%time

data_gpu[arbitrary_features] = ScalerGPU.fit_transform(data_gpu[arbitrary_features].values)
data_gpu["Amount"] = ScalerGPU.fit_transform(data_gpu["Amount"].values.reshape(-1, 1)).squeeze()

CPU times: user 17 s, sys: 766 ms, total: 17.8 s
Wall time: 17.6 s


### Train/Test Split (CPU vs. GPU)

In [ ]:
from sklearn.model_selection import train_test_split as splitCPU
from cuml.preprocessing import train_test_split as splitGPU

In [ ]:
# TODO: Tentukan X (features) dan Y (target), gunakan "data_gpu"
X = data_gpu.iloc[:, :-1]
Y = data_gpu.iloc[:, -1]

print("X_features shape :", X.shape)
print("Y_target shape: ", Y.shape)

X_features shape : (568630, 29)
Y_target shape:  (568630,)


In [ ]:
%%time

# TODO: Pecah dataset dengan komposisi 80% train set dan 20% test set, dengan fungsi "splitCPU"
test_size = 0.2
random_state = 42
x_train, x_test, y_train, y_test = splitCPU(X.to_pandas(), Y.to_pandas(), test_size=test_size, random_state=random_state)

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (454904, 29)
x_test shape:  (113726, 29)
CPU times: user 384 ms, sys: 315 ms, total: 699 ms
Wall time: 718 ms


In [ ]:
%%time

# TODO: Lakukan hal yang sama untuk data spliting, tetapi dengan fungsi "splitGPU"
test_size = 0.2
random_state = 42
x_train, x_test, y_train, y_test = splitCPU(X.to_pandas(), Y.to_pandas(), test_size=test_size, random_state=random_state)

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (454904, 29)
x_test shape:  (113726, 29)
CPU times: user 214 ms, sys: 486 ms, total: 700 ms
Wall time: 679 ms


### Convert the dataset into Tensor

In [ ]:
import cupy # Numpy for GPU

In [ ]:
# TODO: Aktifkan GPU (CUDA) sebagai device untuk training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
x_train_tensor = torch.from_numpy(cupy.asnumpy(x_train)).to(device)
y_train_tensor = torch.from_numpy(cupy.asnumpy(y_train)).to(device)

x_test_tensor = torch.from_numpy(cupy.asnumpy(x_test)).to(device)
y_test_tensor = torch.from_numpy(cupy.asnumpy(y_test)).to(device)

Train_tensor = TensorDataset(x_train_tensor, y_train_tensor)
Test_tensor = TensorDataset(x_test_tensor, y_test_tensor)

print ("Train Tensor Shape: ", Train_tensor.tensors[0].shape, Train_tensor.tensors[1].shape)
print ("Test Tensor Shape: ", Test_tensor.tensors[0].shape, Test_tensor.tensors[1].shape)

Train Tensor Shape:  torch.Size([454904, 29]) torch.Size([454904])
Test Tensor Shape:  torch.Size([113726, 29]) torch.Size([113726])


### Batching the Dataset with PyTorch DataLoader

In [ ]:
# TODO: Tentukan nilai batch
batch_size = 64

Train_dataset = DataLoader(Train_tensor, batch_size=batch_size, shuffle=True)
Test_dataset = DataLoader(Test_tensor, batch_size=batch_size, shuffle=False)

for x_data, y_data in Train_dataset:
  print("X_data shape: ", x_data.shape)
  print("Y_data shape: ", y_data.shape)
  break

X_data shape:  torch.Size([64, 29])
Y_data shape:  torch.Size([64])


### Model Blueprint

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, input_dim, num_neurons):
        super(FeedForward, self).__init__()
        self.input_dim = input_dim
        self.num_neurons = num_neurons

        self.net = nn.Sequential(
            nn.Linear(self.input_dim, self.num_neurons),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

    def to(self, device):
        self.net.to(device)
        return self

class Net(nn.Module):
    def __init__(self, in_features, num_layers, num_neurons):
        super(Net, self).__init__()
        self.in_features = in_features
        self.num_layers = num_layers
        self.num_neurons = num_neurons

        self.fc1 = nn.Linear(self.in_features, self.num_neurons)
        self.relu = nn.ReLU()
        self.blocks = [FeedForward(self.num_neurons, self.num_neurons).to(device) \
                       for _ in range(self.num_layers)]
        self.output_layer = nn.Linear(self.num_neurons, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output = self.relu(self.fc1(x))

        for block in self.blocks:
            output = block(output)
        output = self.sigmoid(self.output_layer(output))

        return output

### Model Hyperparameters and Parameters

In [ ]:
# [ PERTANYAAN ]
# Apa perbedaan hyperparameters dan parameters?

Hyperparameters dan parameters adalah dua konsep penting dalam pembelajaran mesin, dan keduanya memiliki peran yang berbeda dalam membangun model.

- Parameter adalah nilai-nilai internal yang dioptimalkan oleh model selama proses pelatihan. Contohnya adalah bobot dan bias pada neural network.Parameter diubah dan diperbarui setiap kali model melihat data baru dalam proses pelatihan, sehingga model dapat memprediksi dengan lebih akurat.
-Hyperparameters adalah nilai-nilai yang ditetapkan sebelum pelatihan dimulai. Mereka mengontrol bagaimana model belajar, tetapi tidak diubah selama proses pelatihan. Hyperparameters harus ditentukan oleh pengguna (manusia) dan dapat disesuaikan untuk mendapatkan kinerja model yang lebih baik. Pengguna sering kali mencoba berbagai kombinasi hyperparameters untuk "fine-tuning" model.

In [ ]:
# TODO: Tentukan hyperparameters
epochs = 50
num_layers = 3
num_neurons = 64
learning_rate = 0.001

In [ ]:
# TODO: Tentukan besaran input untuk model
# num_inputs = len(arbitrary_features) + 1
num_inputs = 29

model = Net(in_features=num_inputs, num_layers=num_layers, num_neurons=num_neurons)
model = model.to(device)

In [ ]:
# Set the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [ ]:
# Check the number of parameters
print("Number of parameters: {:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
print("Number of trainable parameters: {:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

Number of parameters: 1,985
Number of trainable parameters: 1,985


In [ ]:
# [ PERTANYAAN ]
# Mengapa total "trainable parameters" sama dengan total keseluruhan parameter?

Total "trainable parameters" sama dengan total keseluruhan parameter karena dalam model neural network yang sedang dibangun, semua parameter yang ada di dalam model diatur untuk dapat dioptimalkan selama proses pelatihan. Setiap parameter, seperti bobot dan bias, termasuk dalam kategori trainable karena mereka diperbarui secara bertahap melalui proses backpropagation. Jika ada parameter yang tidak ingin dilatih, biasanya kita harus secara eksplisit menonaktifkannya, misalnya dengan mengatur atribut requires_grad=False, tetapi dalam kasus ini, semua parameter disiapkan untuk dilatih sehingga totalnya sama.

### Train the Model

In [ ]:
print("Start training ...")
for epoch in range(epochs):
    train_loss = 0.0
    model.train()

    for data, label in Train_dataset:
        data = data.to(device)
        label = label.squeeze()
        label = label.to(device)
        optimizer.zero_grad()
        output = model.forward(data.float())

        loss = criterion(output.squeeze(), label.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss = train_loss / len(Train_dataset.dataset)
    if(epoch % 10 == 0):
        print('  - Epoch: {} \tTraining_loss: {:.6f}'.format(epoch, train_loss))

Start training ...
  - Epoch: 0 	Training_loss: 0.001613
  - Epoch: 10 	Training_loss: 0.000044
  - Epoch: 20 	Training_loss: 0.000020
  - Epoch: 30 	Training_loss: 0.000013
  - Epoch: 40 	Training_loss: 0.000009


### Model ACCURACY should reach >= 95%

In [ ]:
# TODO: Jika akurasi masih dibawah 95%, silakan lakukan fine-tuning

In [ ]:
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for data, labels in Test_dataset:
        labels = labels.squeeze()
        output = model.forward(data.float())
        output = output.squeeze(1)

        predictions = (output >= 0.5).float()
        correct_preds += (predictions == labels).sum().item()
        total_samples += labels.numel()

accuracy = correct_preds / total_samples
print("Model accuracy: {:.2f}%".format(accuracy*100))

Model accuracy: 99.96%


### Scoring
Total `#TODO` = 12
<br>Checklist:

- [ ] Impor dataset dengan Pandas, gunakan fungsi "read_by_CPU"
- [ ] Hilangkan kolom ID
- [ ] Tentukan X (features) dan Y (target), gunakan "data_gpu"
- [ ] Pecah dataset dengan komposisi 80% train set dan 20% test set, dengan fungsi "splitCPU"
- [ ] Lakukan hal yang sama untuk data spliting, tetapi dengan fungsi "splitGPU"
- [ ] Aktifkan GPU (CUDA) sebagai device untuk training
- [ ] Tentukan nilai batch
- [ ] PERTANYAAN: Apa perbedaan hyperparameters dan parameters?
- [ ] Tentukan hyperparameters
- [ ] Tentukan besaran input untuk model
- [ ] PERTANYAAN: Mengapa total "trainable parameters" sama dengan total keseluruhan parameter?
- [ ] Jika akurasi masih dibawah 95%, silakan lakukan fine-tuning

### Additional readings
- N/A

### Copyright © 2024 Startup Campus, Indonesia
* Prepared by **Nicholas Dominic, M.Kom.** [(profile)](https://linkedin.com/in/nicholas-dominic)
* You may **NOT** use this file except there is written permission from PT. Kampus Merdeka Belajar (Startup Campus).
* Please address your questions to mentors.